In [ ]:
import joblib
import pickle
import pandas as pd
from datetime import datetime

# 🎯 Load trained model and encoders
rf_model = pickle.load(open("/content/drive/MyDrive/Flight_Dataset/rf_model.pkl", "rb"))
airline_le = joblib.load("/content/drive/MyDrive/Flight_Dataset/airline_label_encoder.pkl")
city_le = joblib.load("/content/drive/MyDrive/Flight_Dataset/city_label_encoder.pkl")

def display_encoded_values():
    print("\n📌 Airline Label Encodings:")
    for i, name in enumerate(airline_le.classes_):
        print(f"{i}: {name}")

    print("\n🏙️ City Label Encodings:")
    for i, name in enumerate(city_le.classes_):
        print(f"{i}: {name}")

def get_user_input():
    display_encoded_values()
    print("\n📥 Enter corresponding integer codes as shown above:")

    airline_enc = int(input("Enter Airline Code (int): "))
    from_enc = int(input("Enter From City Code (int): "))
    to_enc = int(input("Enter To City Code (int): "))

    dep_time = input("Departure Time (HH:MM 24hr format): ").strip()
    arr_time = input("Arrival Time (HH:MM 24hr format): ").strip()
    travel_date = input("Travel Date (YYYY-MM-DD): ").strip()
    time_taken = input("Flight Duration (e.g. 2h 50m): ").strip()

    # 🧠 Time conversion
    dep_hour, dep_min = map(int, dep_time.split(":"))
    arr_hour, arr_min = map(int, arr_time.split(":"))

    # ⏱️ Duration to minutes
    duration = 0
    if "h" in time_taken:
        h_part = int(time_taken.split("h")[0].strip())
        duration += h_part * 60
        if "m" in time_taken:
            m_part = time_taken.split("h")[1].replace("m", "").strip()
            if m_part:
                duration += int(m_part)
    elif "m" in time_taken:
        duration += int(time_taken.replace("m", "").strip())

    # 📅 Extract date components
    date_obj = datetime.strptime(travel_date, "%Y-%m-%d")
    day = date_obj.day
    month = date_obj.month
    weekday = date_obj.weekday()

    # 🧾 Final DataFrame
    input_df = pd.DataFrame([{
        "airline": airline_enc,
        "from": from_enc,
        "to": to_enc,
        "dep_hour": dep_hour,
        "dep_min": dep_min,
        "arr_hour": arr_hour,
        "arr_min": arr_min,
        "duration_mins": duration,
        "day": day,
        "month": month,
        "weekday": weekday
    }])

    return input_df

def predict_price():
    print("\n🛫 Welcome to Flight Price Predictor 🧮")
    input_df = get_user_input()
    prediction = rf_model.predict(input_df)
    print(f"\n💰 Predicted Flight Price: ₹{round(prediction[0], 2)}")

if __name__ == "__main__":
    predict_price()



🛫 Welcome to Flight Price Predictor 🧮

📌 Airline Label Encodings:
0: Air India
1: AirAsia
2: GO FIRST
3: Indigo
4: SpiceJet
5: StarAir
6: Trujet
7: Vistara

🏙️ City Label Encodings:
0: Bangalore
1: Chennai
2: Delhi
3: Hyderabad
4: Kolkata
5: Mumbai

📥 Enter corresponding integer codes as shown above:
Enter Airline Code (int): 7
Enter From City Code (int): 5
Enter To City Code (int): 2
Departure Time (HH:MM 24hr format): 22:00
Arrival Time (HH:MM 24hr format): 04:00
Travel Date (YYYY-MM-DD): 2024-06-06
Flight Duration (e.g. 2h 50m): 3h 50m


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- airline
Feature names seen at fit time, yet now missing:
- airline_encoded


In [ ]:

import joblib
import pickle
import pandas as pd
from datetime import datetime


rf_model = pickle.load(open("/content/drive/MyDrive/Flight_Dataset/rf_model.pkl", "rb"))
airline_le = joblib.load("/content/drive/MyDrive/Flight_Dataset/airline_label_encoder.pkl")
city_le = joblib.load("/content/drive/MyDrive/Flight_Dataset/city_label_encoder.pkl")

# Required column order as per training fitting order only
required_columns = [
    "from", "to", "day", "month", "weekday",
    "duration_mins", "dep_hour", "dep_min",
    "arr_hour", "arr_min", "airline_encoded"
]

def display_encoded_values():
    print("\n Airline Label Encodings:")
    for i, name in enumerate(airline_le.classes_):
        print(f"{i}: {name}")

    print("\n City Label Encodings:")
    for i, name in enumerate(city_le.classes_):
        print(f"{i}: {name}")

def get_user_input():
    display_encoded_values()
    print("\n📥 Enter corresponding integer codes exactly as shown above:\n")

    airline_enc = int(input("Airline Code (int): "))
    from_enc = int(input("From City Code (int): "))
    to_enc = int(input("To City Code (int): "))

    dep_time = input("Departure Time (HH:MM 24hr format): ").strip()
    arr_time = input("Arrival Time (HH:MM 24hr format): ").strip()
    travel_date = input("Travel Date (YYYY-MM-DD): ").strip()
    time_taken = input("Flight Duration (e.g. 2h 50m): ").strip()

    #Time parsing
    dep_hour, dep_min = map(int, dep_time.split(":"))
    arr_hour, arr_min = map(int, arr_time.split(":"))

    #Duration to minutes
    duration = 0
    if "h" in time_taken:
        h_part = int(time_taken.split("h")[0].strip())
        duration += h_part * 60
        if "m" in time_taken:
            m_part = time_taken.split("h")[1].replace("m", "").strip()
            if m_part:
                duration += int(m_part)
    elif "m" in time_taken:
        duration += int(time_taken.replace("m", "").strip())

    #Extract date features
    date_obj = datetime.strptime(travel_date, "%Y-%m-%d")
    day = date_obj.day
    month = date_obj.month
    weekday = date_obj.weekday()

    #Input dictionary
    input_data = {
        "from": from_enc,
        "to": to_enc,
        "day": day,
        "month": month,
        "weekday": weekday,
        "duration_mins": duration,
        "dep_hour": dep_hour,
        "dep_min": dep_min,
        "arr_hour": arr_hour,
        "arr_min": arr_min,
        "airline_encoded": airline_enc
    }

    # 🧾 Build ordered input DataFrame
    input_df = pd.DataFrame([[input_data[col] for col in required_columns]], columns=required_columns)
    return input_df

def predict_price():
    print("\n🛫 Welcome to Flight Price Predictor 🧮")
    input_df = get_user_input()
    prediction = rf_model.predict(input_df)
    print(f"\n💰 Predicted Flight Price: ₹{round(prediction[0], 2)}")

if __name__ == "__main__":
    predict_price()



🛫 Welcome to Flight Price Predictor 🧮

📌 Airline Label Encodings:
0: Air India
1: AirAsia
2: GO FIRST
3: Indigo
4: SpiceJet
5: StarAir
6: Trujet
7: Vistara

🏙️ City Label Encodings:
0: Bangalore
1: Chennai
2: Delhi
3: Hyderabad
4: Kolkata
5: Mumbai

📥 Enter corresponding integer codes exactly as shown above:

Airline Code (int): 7
From City Code (int): 5
To City Code (int): 2
Departure Time (HH:MM 24hr format): 22:00
Arrival Time (HH:MM 24hr format): 04:00
Travel Date (YYYY-MM-DD): 2024-06-06
Flight Duration (e.g. 2h 50m): 3h 50m

💰 Predicted Flight Price: ₹4556.44


In [ ]:
#Requirements: airline_label_encoder.pkl, city_label_encoder.pkl, economy.csv, rf_model.pkl